<a href="https://colab.research.google.com/github/gwangjinjeong/Pytorch/blob/master/basic/04_Softmax_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Softmasx Classification

* What is Softmax
* Cross Entropy
* Low-level Implementation
* High-level Implementation
* Training Example

# *0. Preparation*

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

+) Discrete Probability distribution (이산확률분포)

조건부 확률   
P(주먹|가위) => 가위를 냈을때, 주먹을 낼 확률값.   
P(가위|가위) => 가위를 냈을때, 가위을 낼 확률값.   
P(보|가위) => 가위를 냈을때, 보을 낼 확률값.   



# *1. Softmax*

max값을 soft하게 뽑아줘

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [0]:
z = torch.FloatTensor([1, 2, 3])

여기서 amx를 뽑는다면 (0, 0, 1)으로 나오게 된다.

하지만 `softmax` 는,

In [0]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


위와 같이 `마지막 3의 값이 66%로 max값이다.`와 같이 확률값으로 나타나게 한다.

In [0]:
hypothesis.sum()

tensor(1.)

# *2. Cross Entropy*

$ H(P, Q) = -E_{x\sim P(x)}[ logQ(x) ]$ = $- \sum_{x\in X}P(x)logQ(x)$   


가위바위보 게임에서, $Q_1$이 바위, $Q_2$가 보라고 가정하고, 
상대방이 가위를 낼 확률이 P라고 했을때, 상대방를 이기기 위해서
H(P, Q)를 최소화 하도록 한다면, 점점 Q가 P에 가까워질 것이므로, 어떤 값을 낼지 예측할 수 있다.

## 2.1 Cross Entropy Loss (Low-level)

다중 분류를 하는데 사용되는 loss를 Cross Entropy loss라고 한다.   
어떻게 최소화 하는가?

$ L = \frac{1}{N} \sum - y \log(\hat{y}) $   
y는 실제 값, P(X)을 의미한다.   
$P_\theta$ $\theta$를 낼 확률을 의미한다.

예)
클래스 갯수를 5개   
sample 갯수를 3개로 만들어보자

In [3]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) # 즉, 두번째 dimantion이 결과값이다. 즉, y^ d이다.
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [4]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [5]:
y_one_hot = torch.zeros_like(hypothesis) # (3,5)를 만들어줘라.
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim 1에 대해서 unsqueeze를 해서 1을 뿌려줘라.
# y의 size는 (3,) => y.unsqueeze는 (3,1)로 되었다.

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [9]:
print((y_one_hot * -torch.log(hypothesis)))
print((y_one_hot * -torch.log(hypothesis)).sum(dim=1))
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor([[1.3301, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4602, 0.0000, 0.0000],
        [0.0000, 1.6165, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>)
tensor([1.3301, 1.4602, 1.6165], grad_fn=<SumBackward1>)
tensor(1.4689, grad_fn=<MeanBackward0>)


## 2.2 Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [11]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [12]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [13]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [10]:
# High level NLL = Negative Log Likelihood
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [14]:
# nll_loss + log_softmax
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

일반적으로 확률 값을 봐야하는 경우가 많기 때문에,   
Neural network에서는 softmax 이전 값을 가져와야하기도 한다.

## 2.3 High-level Implementation with


Make the Train set

In [0]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # Discrete 한 값. 
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [16]:
# 모델 초기화
# vector size(columns)  = 4
# number of sample(row) = m
# classes               = 3
W = torch.zeros((4, 3), requires_grad=True) 
b = torch.zeros(1, requires_grad=True)      
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    # 물론 위 3개를     cost = F.cross_entropy(z, y_train)로 만들어줄 수도 있다.

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


## 2.4 High-level Implementation with `nn.Module`

In [0]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3(class 갯수)

    def forward(self, x):
        return self.linear(x) # |x| = (m,4) => (m,3)

In [0]:
model = SoftmaxClassifierModel()

Let's try another new dataset.

In [19]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train) # |x_train| = (m,4), |prediction| = (m,3) 

    # cost 계산
    cost = F.cross_entropy(prediction, y_train) #|y_train | = (m,)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922


**1과 0으로 이루어진 binary classification(이진분류)**   
binary classification는 logistic regression과 굉장히 유사한데, 그래서 binary classification에서는 binary corss entorpy loss를 사용하는것이 맞다.
그리고 sigmoid를 써야한다.      

**2개 이상의 multi-class로 이루어진 분류**   
이때는, Cross-entropy를 사용하고, softmax를 사용해야한다.
